In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [3]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
padded_shapes = ([None], ())

In [4]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
trainset, testset = dataset['train'], dataset['test']
encoder = info.features['text'].encoder
trainset = trainset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE,
                                         padded_shapes=padded_shapes)
testset = testset.padded_batch(BATCH_SIZE,
                               padded_shapes=padded_shapes)


In [5]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size, 64),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
                             ])

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(trainset, epochs=5, validation_data=testset,
                    validation_steps=30)

Epoch 1/5
391/391 [==============================] - 53s 112ms/step - loss: 0.6799 - accuracy: 0.5443 - val_loss: 0.5028 - val_accuracy: 0.7406
Epoch 2/5
391/391 [==============================] - 44s 110ms/step - loss: 0.3743 - accuracy: 0.8431 - val_loss: 0.3582 - val_accuracy: 0.8615
Epoch 3/5
391/391 [==============================] - 43s 110ms/step - loss: 0.2594 - accuracy: 0.9021 - val_loss: 0.3485 - val_accuracy: 0.8625
Epoch 4/5
391/391 [==============================] - 43s 109ms/step - loss: 0.2211 - accuracy: 0.9218 - val_loss: 0.3438 - val_accuracy: 0.8672
Epoch 5/5
391/391 [==============================] - 43s 109ms/step - loss: 0.1869 - accuracy: 0.9354 - val_loss: 0.3742 - val_accuracy: 0.8646


In [7]:
def pad_to_size(vec, size):
  zeros = [0]*(size - len(vec))
  vec.extend(zeros)
  return vec

In [21]:
def sample_predict(sentence, pad, model_):
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)

  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model_.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return predictions

In [15]:
# Dummy Reviews
sample_text_pos = ('This movie was so awesome. Acting was really nice, I recommend it')
predictions1 = sample_predict(sample_text_pos, pad=True, model_=model) * 100
print('Probability of Positive review %.2f' % predictions1)
sample_text_neg = ('I did not like the movie. The acting was okayish')
predictions2 = sample_predict(sample_text_neg, pad=True, model_=model) * 100
print('Probability of Positive review %.2f' % predictions2)

Probability of Positive review 62.91
Probability of Positive review 26.84


Now let's make a more complex model

In [16]:
model2 = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size, 64),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                              tf.keras.layers.Dense(64, activation='relu'),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(1, activation='sigmoid')
                              ])

In [19]:
model2.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy',
              metrics=['accuracy'])
history2 = model2.fit(trainset, epochs=5, validation_data=testset,
                    validation_steps=30)

Epoch 1/5
391/391 [==============================] - 81s 194ms/step - loss: 0.6757 - accuracy: 0.5407 - val_loss: 0.4345 - val_accuracy: 0.8073
Epoch 2/5
391/391 [==============================] - 75s 189ms/step - loss: 0.3594 - accuracy: 0.8572 - val_loss: 0.3711 - val_accuracy: 0.8490
Epoch 3/5
391/391 [==============================] - 74s 187ms/step - loss: 0.2635 - accuracy: 0.9057 - val_loss: 0.3619 - val_accuracy: 0.8521
Epoch 4/5
391/391 [==============================] - 73s 186ms/step - loss: 0.2122 - accuracy: 0.9280 - val_loss: 0.3917 - val_accuracy: 0.8630
Epoch 5/5
391/391 [==============================] - 74s 188ms/step - loss: 0.1835 - accuracy: 0.9407 - val_loss: 0.3993 - val_accuracy: 0.8599


In [23]:
# Dummy Reviews
sample_text_pos = ('This movie was so awesome. Acting was really nice, I recommend it')
predictions1 = sample_predict(sample_text_pos, pad=True, model_=model2) * 100
print('Probability of Positive review %.2f' % predictions1)
sample_text_neg = ('I did not like the movie. The acting was okayish')
predictions2 = sample_predict(sample_text_neg, pad=True, model_=model2) * 100
print('Probability of Positive review %.2f' % predictions2)

Probability of Positive review 88.10
Probability of Positive review 23.42
